<a href="https://colab.research.google.com/github/endnef2/big_leader/blob/main/DL/Pytorch_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Pytroch Autograd

In [1]:
import torch

In [2]:
torch.__version__

'2.0.1+cu118'

In [3]:
x = torch.randn(3)
y = x+2

In [4]:
x

tensor([ 1.0905,  0.9467, -0.7810])

In [5]:
y

tensor([3.0905, 2.9467, 1.2190])

In [6]:
z = y*y*3

In [7]:
# z의 평균값
z.mean()

tensor(19.7201)

In [8]:
z.mean().item()

19.720109939575195

In [9]:
a = torch.rand(2,2,requires_grad=True)
a

tensor([[0.8292, 0.6482],
        [0.4802, 0.4019]], requires_grad=True)

In [10]:
a.requires_grad

True

In [11]:
# requires_gard 빼고 tensor valure 값만
a.detach()

tensor([[0.8292, 0.6482],
        [0.4802, 0.4019]])

In [12]:
# torch.no_gard()
with torch.no_grad():
  print((x**2).requires_grad)

False


In [13]:
# torch.no_gard()
with torch.no_grad():
  print((x**2))

tensor([1.1891, 0.8963, 0.6100])


In [14]:
weight = torch.ones(4, requires_grad=True)

# 학습시킬 때 파이토치는 for 문 사용
for epoch in range(5):
  model_out = (weight*3).sum()
  model_out.backward()

  print(weight.grad)

  with torch.no_grad():
    weight-=0.1*weight.grad # weight 조정 weight = weight-0.1(learning rate)*weight.grad

  weight.grad.zero_()

print(weight)
print(model_out)

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([-0.5000, -0.5000, -0.5000, -0.5000], requires_grad=True)
tensor(-2.4000, grad_fn=<SumBackward0>)


## 2. Backpropagation

In [15]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)

In [16]:
w = torch.tensor(1.0, requires_grad=True)

In [17]:
y_predict = w * x

In [18]:
loss = (y-y_predict)**2

In [19]:
print(loss)

tensor(1., grad_fn=<PowBackward0>)


In [20]:
loss.backward()
w.grad

tensor(-2.)

In [21]:
with torch.no_grad():
  w -= 0.01 *w.grad

# 값을 0으로 바꿔라
w.grad.zero_()

tensor(0.)

## 3. Gradient Descent: Manually

In [22]:
import numpy as np

X = np.array([1,2,3,4], dtype=np.float32)
Y = np.array([2,4,6,8], dtype=np.float32)

w = 0.0 # 실수로 넣어주기

In [23]:
def forward(x):
  return w*x

In [24]:
def loss(y, y_pred):
  return ((y_pred-y)**2).mean()

In [25]:
def gradient(x, y, y_pred):
  return np.dot(2*x, y_pred-y).mean()
print(forward(5))

0.0


In [26]:
# training
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
  # forward pass
  y_pred = forward(X)

  # loss
  l = loss(Y, y_pred)

  # gradients 계산
  dw = gradient(X, Y, y_pred) # input, output, 예측값

  # update weight
  w -= learning_rate * dw

  if epoch%2==0:
    print(f'epoch {epoch+1}: w = {w:.3f}, loss={l:.4f}')
print(f'predict after training: f(5)={forward(5):.3f}')

epoch 1: w = 1.200, loss=30.0000
epoch 3: w = 1.872, loss=0.7680
epoch 5: w = 1.980, loss=0.0197
epoch 7: w = 1.997, loss=0.0005
epoch 9: w = 1.999, loss=0.0000
epoch 11: w = 2.000, loss=0.0000
epoch 13: w = 2.000, loss=0.0000
epoch 15: w = 2.000, loss=0.0000
epoch 17: w = 2.000, loss=0.0000
epoch 19: w = 2.000, loss=0.0000
predict after training: f(5)=10.000


## 4. Gradient Descent : Autograd

In [27]:
import torch

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

In [28]:
def forward(x):
  return w*x

def loss(y, y_pred):
  return ((y_pred-y)**2).mean()

In [29]:
# torch training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
  # forward predict
  y_pred = forward(X)

  # loss
  l = loss(Y, y_pred)

  # backward pass
  l.backward()

  # update weights
  # w.data = w.data - learning_rate*w.grad
  with torch.no_grad():
    w -= learning_rate*w.grad

  # update 후에 w를 zero화 시킨다.
  w.grad.zero_()

  if epoch % 10 == 0:
    print(f'epoch:{epoch+1}:w={w.item():.3f},loss={l.item():4f}')
print(f'Prediction after training: f(5)={forward(5).item():.3f}')

epoch:1:w=0.300,loss=30.000000
epoch:11:w=1.665,loss=1.162786
epoch:21:w=1.934,loss=0.045069
epoch:31:w=1.987,loss=0.001747
epoch:41:w=1.997,loss=0.000068
epoch:51:w=1.999,loss=0.000003
epoch:61:w=2.000,loss=0.000000
epoch:71:w=2.000,loss=0.000000
epoch:81:w=2.000,loss=0.000000
epoch:91:w=2.000,loss=0.000000
Prediction after training: f(5)=10.000


## 5. Loss Function & Optimizer

In [30]:
import torch
import torch.nn as nn

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

In [31]:
def forward(x):
  return w*x

In [32]:
# hyper parameter
learning_rate = 0.01
n_iters = 100

In [33]:
# loss function and optimizer
loss = nn.MSELoss()
optimizer = torch.optim.SGD([w], lr=learning_rate)

In [34]:
for epoch in range(n_iters):
  # forward predict
  y_pred = forward(X)

  # loss
  l = loss(Y, y_pred)

  # backward pass
  l.backward()

  # update weights
  optimizer.step()

  # update 후에 w를 zero화 시킨다.
  optimizer.zero_grad()

  if epoch % 10 == 0:
    print('epoch', epoch+1, ':w=', w, 'loss=', l)

print(f'Prediction after training: f(5)={forward(5).item():.3f}')

epoch 1 :w= tensor(0.3000, requires_grad=True) loss= tensor(30., grad_fn=<MseLossBackward0>)
epoch 11 :w= tensor(1.6653, requires_grad=True) loss= tensor(1.1628, grad_fn=<MseLossBackward0>)
epoch 21 :w= tensor(1.9341, requires_grad=True) loss= tensor(0.0451, grad_fn=<MseLossBackward0>)
epoch 31 :w= tensor(1.9870, requires_grad=True) loss= tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch 41 :w= tensor(1.9974, requires_grad=True) loss= tensor(6.7705e-05, grad_fn=<MseLossBackward0>)
epoch 51 :w= tensor(1.9995, requires_grad=True) loss= tensor(2.6244e-06, grad_fn=<MseLossBackward0>)
epoch 61 :w= tensor(1.9999, requires_grad=True) loss= tensor(1.0176e-07, grad_fn=<MseLossBackward0>)
epoch 71 :w= tensor(2.0000, requires_grad=True) loss= tensor(3.9742e-09, grad_fn=<MseLossBackward0>)
epoch 81 :w= tensor(2.0000, requires_grad=True) loss= tensor(1.4670e-10, grad_fn=<MseLossBackward0>)
epoch 91 :w= tensor(2.0000, requires_grad=True) loss= tensor(5.0768e-12, grad_fn=<MseLossBackward0>)
Prediction

## 6. Advanced Training

In [35]:
# 1차원이 아닌 2차원으로 넣어야 error가 나지 않는다.
import torch
import torch.nn as nn

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

n_samples, n_features = X.shape

In [36]:
X_test = torch.tensor([5], dtype=torch.float32)
X_test

tensor([5.])

In [37]:
input_size = n_features
output_size = Y.shape[1]

In [38]:
# model build
model = nn.Linear(input_size, output_size)

In [39]:
class LinearRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
      super(LinearRegression, self).__init__()
      # define Layers
      self.lin = nn.Linear(input_dim, output_dim)

  def forward(self, x):
      return self.lin(x)

In [40]:
model = LinearRegression(input_size, output_size)

In [41]:
learning_rate = 0.01
n_iters=100

# loss function and optimizer
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [42]:
for epoch in range(n_iters):
  # forward predict
  y_pred = model(X)

  # loss
  l = loss(Y, y_pred)

  # backward pass
  l.backward()

  # update weights
  optimizer.step()

  # update 후에 w를 zero화 시킨다.
  optimizer.zero_grad()

  if epoch % 10 == 0:
    w, b = model.parameters()
    print('epoch', epoch+1, ':w=', w[0].item(), 'loss=', l)

print(f'Prediction after training: f(5)={model(X_test).item():.3f}')

epoch 1 :w= 0.05526984855532646 loss= tensor(33.1790, grad_fn=<MseLossBackward0>)
epoch 11 :w= 1.3814436197280884 loss= tensor(1.0481, grad_fn=<MseLossBackward0>)
epoch 21 :w= 1.6037887334823608 loss= tensor(0.2058, grad_fn=<MseLossBackward0>)
epoch 31 :w= 1.6483174562454224 loss= tensor(0.1736, grad_fn=<MseLossBackward0>)
epoch 41 :w= 1.6639856100082397 loss= tensor(0.1630, grad_fn=<MseLossBackward0>)
epoch 51 :w= 1.674760341644287 loss= tensor(0.1535, grad_fn=<MseLossBackward0>)
epoch 61 :w= 1.6845040321350098 loss= tensor(0.1445, grad_fn=<MseLossBackward0>)
epoch 71 :w= 1.6938453912734985 loss= tensor(0.1361, grad_fn=<MseLossBackward0>)
epoch 81 :w= 1.7028924226760864 loss= tensor(0.1282, grad_fn=<MseLossBackward0>)
epoch 91 :w= 1.7116690874099731 loss= tensor(0.1207, grad_fn=<MseLossBackward0>)
Prediction after training: f(5)=9.422
